<a href="https://colab.research.google.com/github/haddybhaiya/sem-i-con/blob/main/train_convNeXt_tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm
!pip install scikit-learn


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import timm
import os
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
from tqdm import tqdm


In [ ]:
DATASET_PATH = "/content/drive/MyDrive/synthetic_dataset"   # change if needed
NUM_CLASSES = 8
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

CLASSES = [
    "clean","bridge","cmp","crack","open","ler","via","other"
]


In [ ]:
class SEMDataset(Dataset):
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, cls = self.samples[idx]

        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img.astype(np.float32) / 255.0

        img = np.expand_dims(img, axis=0)

        label = CLASSES.index(cls)

        return torch.tensor(img), torch.tensor(label)


In [ ]:
from sklearn.model_selection import train_test_split

all_samples = []

for cls in CLASSES:
    folder = os.path.join(DATASET_PATH, cls)

    for img in os.listdir(folder):
        all_samples.append((os.path.join(folder, img), cls))

train_samples, val_samples = train_test_split(
    all_samples,
    test_size=0.2,
    stratify=[s[1] for s in all_samples],   # keeps class balance
    random_state=42
)

print("Train:", len(train_samples))
print("Val:", len(val_samples))


Train: 1920
Val: 480


In [ ]:
train_dataset = SEMDataset(train_samples)
val_dataset   = SEMDataset(val_samples)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE)


In [ ]:
model = build_convnext().to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for x, y in tqdm(train_loader):
        x, y = x.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} Loss:", total_loss / len(train_loader))


100%|██████████| 120/120 [21:58<00:00, 10.99s/it]


Epoch 1 Loss: 0.25449785136200564


100%|██████████| 120/120 [00:41<00:00,  2.89it/s]


Epoch 2 Loss: 0.03702565324122891


100%|██████████| 120/120 [00:40<00:00,  2.95it/s]


...

Epoch 18 Loss: 5.684935202528626e-06


100%|██████████| 120/120 [00:40<00:00,  2.93it/s]


Epoch 19 Loss: 5.126411087985616e-06


100%|██████████| 120/120 [00:41<00:00,  2.92it/s]

Epoch 20 Loss: 4.630343515070005e-06


In [ ]:
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for x, y in val_loader:
        x = x.to(DEVICE)

        out = model(x)
        preds = torch.argmax(out, dim=1).cpu()

        all_preds.extend(preds.numpy())
        all_labels.extend(y.numpy())

print(classification_report(all_labels, all_preds, target_names=CLASSES))


              precision    recall  f1-score   support

       clean       1.00      1.00      1.00        60
      bridge       1.00      1.00      1.00        60
         cmp       1.00      1.00      1.00        60
       crack       1.00      1.00      1.00        60
        open       1.00      1.00      1.00        60
         ler       1.00      1.00      1.00        60
         via       1.00      1.00      1.00        60
       other       1.00      1.00      1.00        60

    accuracy                           1.00       480
   macro avg       1.00      1.00      1.00       480
weighted avg       1.00      1.00      1.00       480



In [ ]:
torch.save(model.state_dict(), "convnext_sem.pth")
print("Model saved")


Model saved


In [ ]:
!pip install onnxruntime
!pip install onnx
!pip install onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 15.4 MB/s eta 0:00:00


In [ ]:
import torch
import timm
import torch.nn as nn
import os

MODEL_PATH = "/content/convnext_sem.pth"
ONNX_PATH = "sem.onnx"

NUM_CLASSES = 8
IMG_SIZE = 224
DEVICE = "cpu"

CLASSES = ["clean","bridge","cmp","crack","open","ler","via","other"]

# ---- Build Model ----
model = timm.create_model("convnext_tiny", pretrained=False, in_chans=1)
model.head.fc = nn.Linear(model.head.fc.in_features, NUM_CLASSES)

model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

dummy_input = torch.randn(1, 1, IMG_SIZE, IMG_SIZE)

torch.onnx.export(
    model,
    dummy_input,
    ONNX_PATH,
    input_names=["input"],
    output_names=["output"],
    opset_version=17,
    dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}},
)

print("✅ FP32 ONNX exported:", ONNX_PATH)


/tmp/ipython-input-3497153173.py:24: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0203 10:55:42.174000 800 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W0203 10:55:43.365000 800 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, alig

[torch.onnx] Obtain model graph for `ConvNeXt([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ConvNeXt([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnx/version_converter.py", line 39, in convert_version
    converted_model_str = C.convert_version(model_str, target_version)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: /github/workspace/onnx/version_converter/adapters/axes_input_to_attribute.h:65: adapt: Asserti

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


✅ FP32 ONNX exported: sem.onnx


In [ ]:
import torch
import timm
import torch.nn as nn

MODEL_PATH = "convnext_sem.pth"
OUT_PATH = "sem_int8.pth"
NUM_CLASSES = 8

device = "cpu"

# Build model
model = timm.create_model(
    "convnext_tiny",
    pretrained=False,
    num_classes=NUM_CLASSES,
    in_chans=1
)

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# ---- Dynamic quantization ----
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {nn.Linear},
    dtype=torch.qint8
)

torch.save(quantized_model.state_dict(), OUT_PATH)

print("✅ Torch INT8 model saved:", OUT_PATH)


/tmp/ipython-input-3074260091.py:23: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  quantized_model = torch.quantization.quantize_dynamic(


✅ Torch INT8 model saved: sem_int8.pth


In [ ]:
import torch
import timm

MODEL_PATH = "sem_int8.pth"
OUT_PATH = "sem_int8.pt"
NUM_CLASSES = 8

model = timm.create_model(
    "convnext_tiny",
    pretrained=False,
    num_classes=NUM_CLASSES,
    in_chans=1
)

model.load_state_dict(torch.load(MODEL_PATH, map_location="cpu"))
model.eval()

example = torch.randn(1,1,224,224)

scripted = torch.jit.trace(model, example)
scripted.save(OUT_PATH)

print("✅ TorchScript INT8 exported:", OUT_PATH)


RuntimeError: Error(s) in loading state_dict for ConvNeXt:
	Missing key(s) in state_dict: "stages.0.blocks.0.mlp.fc1.weight", "stages.0.blocks.0.mlp.fc1.bias", "stages.0.blocks.0.mlp.fc2.weight", "stages.0.blocks.0.mlp.fc2.bias", "stages.0.blocks.1.mlp.fc1.weight", "stages.0.blocks.1.mlp.fc1.bias", "stages.0.blocks.1.mlp.fc2.weight", "stages.0.blocks.1.mlp.fc2.bias", "stages.0.blocks.2.mlp.fc1.weight", "stages.0.blocks.2.mlp.fc1.bias", "stages.0.blocks.2.mlp.fc2.weight", "stages.0.blocks.2.mlp.fc2.bias", "stages.1.blocks.0.mlp.fc1.weight", "stages.1.blocks.0.mlp.fc1.bias", "stages.1.blocks.0.mlp.fc2.weight", "stages.1.blocks.0.mlp.fc2.bias", "stages.1.blocks.1.mlp.fc1.weight", "stages.1.blocks.1.mlp.fc1.bias", "stages.1.blocks.1.mlp.fc2.weight", "stages.1.blocks.1.mlp.fc2.bias", "stages.1.blocks.2.mlp.fc1.weight", "stages.1.blocks.2.mlp.fc1.bias", "stages.1.blocks.2.mlp.fc2.weight", "stages.1.blocks.2.mlp.fc2.bias", "stages.2.blocks.0.mlp.fc1.weight", "stages.2.blocks.0.mlp.fc1.bias", 

In [ ]:
import onnxruntime as ort
import cv2
import numpy as np

MODEL_PATH = "models/sem_int8.onnx"

CLASSES = ["clean","bridge","cmp","crack","open","ler","via","other"]

IMG_SIZE = 224
OTHER_THRESHOLD = 0.65

session = ort.InferenceSession(MODEL_PATH, providers=["CPUExecutionProvider"])
input_name = session.get_inputs()[0].name


def preprocess(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img.astype(np.float32) / 255.0

    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=0)

    return img


def softmax(x):
    e = np.exp(x - np.max(x))
    return e / np.sum(e)


def infer(img_path):
    x = preprocess(img_path)
    out = session.run(None, {input_name: x})

    logits = out[0][0]
    probs = softmax(logits)

    cls_id = int(np.argmax(probs))
    conf = float(probs[cls_id])

    pred = CLASSES[cls_id]

    if conf < OTHER_THRESHOLD:
        pred = "other"

    return pred, conf


if __name__ == "__main__":
    img = "dataset/sample/test.png"
    print(infer(img))


In [ ]:
import psutil
import time
import onnxruntime as ort
import numpy as np
import cv2

FP32_MODEL = "models/sem.onnx"
INT8_MODEL = "models/sem_int8.onnx"

CLASSES = ["clean","bridge","cmp","crack","open","ler","via","other"]

IMG_SIZE_HIGH = 224
IMG_SIZE_LOW = 160

CPU_THRESHOLD = 70

session_fp32 = ort.InferenceSession(FP32_MODEL)
session_int8 = ort.InferenceSession(INT8_MODEL)

input_fp32 = session_fp32.get_inputs()[0].name
input_int8 = session_int8.get_inputs()[0].name


def preprocess(img_path, size):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (size, size))
    img = img.astype(np.float32) / 255.0

    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=0)

    return img


def softmax(x):
    e = np.exp(x - np.max(x))
    return e / np.sum(e)


def auto_edge(img_path):
    cpu = psutil.cpu_percent(interval=0.1)

    if cpu > CPU_THRESHOLD:
        session = session_int8
        input_name = input_int8
        size = IMG_SIZE_LOW
        mode = "INT8_LOW"
    else:
        session = session_fp32
        input_name = input_fp32
        size = IMG_SIZE_HIGH
        mode = "FP32_HIGH"

    start = time.time()

    x = preprocess(img_path, size)
    out = session.run(None, {input_name: x})

    latency = time.time() - start

    logits = out[0][0]
    probs = softmax(logits)

    cls_id = int(np.argmax(probs))
    conf = float(probs[cls_id])

    return {
        "class": CLASSES[cls_id],
        "confidence": conf,
        "mode": mode,
        "latency": latency,
        "cpu": cpu
    }


if __name__ == "__main__":
    print(auto_edge("dataset/sample/test.png"))


In [ ]:
from sklearn.metrics import classification_report
import os

y_true = []
y_pred = []

TEST_PATH = "dataset/test"

for cls in CLASSES:
    folder = os.path.join(TEST_PATH, cls)

    for img in os.listdir(folder):
        path = os.path.join(folder, img)

        pred, _ = infer(path)

        y_true.append(cls)
        y_pred.append(pred)

print(classification_report(y_true, y_pred))
